In [9]:
pip install kafka-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


we need to create object for kafka producer

In [3]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [1]:
from kafka import KafkaProducer
import json
import requests
import time

# Simple configuration
producer = KafkaProducer(
    bootstrap_servers=['15.207.54.208:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

processed_ids = set()

print("Starting earthquake producer...")

while True:
    # Get earthquake data
    response = requests.get("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson")
    earthquakes = response.json()['features']
    
    # Send new earthquakes
    for earthquake in earthquakes:
        earthquake_id = earthquake['id']
        
        if earthquake_id not in processed_ids:
            # Simple earthquake data
            data = {
                'id': earthquake_id,
                'magnitude': earthquake['properties']['mag'],
                'place': earthquake['properties']['place'],
                'time': earthquake['properties']['time']
            }
            
            producer.send('earthquakes-events2', data)
            processed_ids.add(earthquake_id)
            print(f"Sent: {earthquake_id} - Mag: {data['magnitude']}")
    
    # Keep memory low
    if len(processed_ids) > 1000:
        processed_ids = set(list(processed_ids)[-500:])
    
    print("Waiting 5 minutes...")
    time.sleep(300)

Starting earthquake producer...


KafkaTimeoutError: KafkaTimeoutError: Failed to update metadata after 60.0 secs.

In [4]:
import requests
from kafka import KafkaProducer
from json import dumps
from time import sleep

producer = KafkaProducer(
    bootstrap_servers=['13.126.75.47:9092'],  # Put your EC2 Public IP
    value_serializer=lambda x: dumps(x).encode('utf-8'),
    request_timeout_ms=30000,
    retry_backoff_ms=500
)

def fetch_api_data():
    
    url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"  # Example earthquake API
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['features']
    return []

while True:
    data = fetch_api_data()
    if data:
        for item in data[:3]:  # Just send 3 items to avoid overloading Kafka
            producer.send('demo_test27', value=item) 
            print("Sent:", item)
            sleep(1)
    else:
        print("No data fetched.")
    sleep(10)  # Pause before next API fetch

producer.flush()

Sent: {'type': 'Feature', 'properties': {'mag': 0.3, 'place': '63 km WNW of Tyonek, Alaska', 'time': 1753611183517, 'updated': 1753611286627, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak0259k5tbcf', 'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/ak0259k5tbcf.geojson', 'felt': None, 'cdi': None, 'mmi': None, 'alert': None, 'status': 'automatic', 'tsunami': 0, 'sig': 1, 'net': 'ak', 'code': '0259k5tbcf', 'ids': ',ak0259k5tbcf,', 'sources': ',ak,', 'types': ',origin,phase-data,', 'nst': None, 'dmin': None, 'rms': 0.11, 'gap': None, 'magType': 'ml', 'type': 'earthquake', 'title': 'M 0.3 - 63 km WNW of Tyonek, Alaska'}, 'geometry': {'type': 'Point', 'coordinates': [-152.244, 61.2633, 6.1]}, 'id': 'ak0259k5tbcf'}
Sent: {'type': 'Feature', 'properties': {'mag': 0.7, 'place': '13 km NNE of Cabazon, CA', 'time': 1753611149100, 'updated': 1753611393589, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci41033503', 'detail': 'h

KeyboardInterrupt: 